In [26]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, MultiPolygon
from shapely import to_geojson

In [27]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [28]:
%sql postgresql://teleport@localhost:59950/property_data

'Connected: teleport@property_data'

In [29]:
multi_geos = pd.read_csv('data/crmls_address_multiple_geos.csv')
multi_geos.shape

(1449, 5)

In [30]:
multi_geos.columns

Index(['ParcelFullAddress', 'GeoJson_Polygon', 'ogr_fid', 'parcelapn',
       'count_geom'],
      dtype='object')

In [31]:
def combine_geos(geo_series):
    poly_list = []
    for i in geo_series:
        if type(i) == MultiPolygon:
            poly_list += i.geoms
        else:
            poly_list.append(i)

    return MultiPolygon(poly_list)

In [32]:
multi_geos = gpd.GeoDataFrame(multi_geos, geometry=[shape(json.loads(x)) for x in multi_geos['GeoJson_Polygon']])

In [33]:
multi_geos.head()

,ParcelFullAddress,GeoJson_Polygon,ogr_fid,parcelapn,count_geom,geometry
0,VILLA RDG IRVINE CA 92620,"{""type"": ""Polygon"", ""coordinates"": [[[-117.754...",695352,551-602-23,184,"POLYGON ((-117.75429 33.68460, -117.75444 33.6..."
1,VILLA RDG IRVINE CA 92620,"{""type"": ""Polygon"", ""coordinates"": [[[-117.753...",695348,551-602-25,184,"POLYGON ((-117.75368 33.68521, -117.75382 33.6..."
2,VILLA RDG IRVINE CA 92620,"{""type"": ""Polygon"", ""coordinates"": [[[-117.754...",695347,551-602-19,184,"POLYGON ((-117.75400 33.68514, -117.75432 33.6..."
3,VILLA RDG IRVINE CA 92620,"{""type"": ""Polygon"", ""coordinates"": [[[-117.754...",695272,551-602-29,184,"POLYGON ((-117.75415 33.68420, -117.75447 33.6..."
4,VILLA RDG IRVINE CA 92620,"{""type"": ""Polygon"", ""coordinates"": [[[-117.754...",695265,551-602-22,184,"POLYGON ((-117.75461 33.68428, -117.75468 33.6..."


In [34]:
all_addresses = set(multi_geos['ParcelFullAddress'])
multi_poly = []
for i in all_addresses:
    m = combine_geos(multi_geos[multi_geos['ParcelFullAddress'] == i]['geometry'])
    multi_poly.append(m)

print(len(all_addresses), len(multi_poly))

453 453


In [35]:
combined_geos = gpd.GeoDataFrame({'ParcelFullAddress': list(all_addresses)}, geometry=multi_poly)
combined_geos.head()

,ParcelFullAddress,geometry
0,2 LIBERTY IRVINE CA 92620,"MULTIPOLYGON (((-117.77023 33.71646, -117.7719..."
1,19 GLORIETA IRVINE CA 92620,"MULTIPOLYGON (((-117.75976 33.71961, -117.7600..."
2,21 ENSUENO IRVINE CA 92620,"MULTIPOLYGON (((-117.76202 33.71880, -117.7623..."
3,4 CAMPHOR IRVINE CA 92612,"MULTIPOLYGON (((-117.79852 33.66294, -117.7988..."
4,22 ENTRADA IRVINE CA 92620,"MULTIPOLYGON (((-117.75535 33.71856, -117.7556..."


In [36]:
combined_geos['geometry'] = to_geojson(combined_geos['geometry'])
combined_geos.to_csv('combined_geos.csv')

/Users/arnavmalhotra/anaconda3/envs/pw/lib/python3.10/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
